In [1]:
import requests


In [8]:
url = 'https://test-pred-maint-projects.onrender.com/list_features'
response = requests.get(url)

# Afficher le statut de la réponse
print('Statut de la réponse:', response.status_code)

# Afficher le contenu de la réponse
print('Contenu de la réponse:', response.text)

Statut de la réponse: 200
Contenu de la réponse: ["Type","Air temperature [K]","Process temperature [K]","Rotational speed [rpm]","Torque [Nm]","Tool wear [min]","Difference temperature [K]","Power"]


In [7]:
response_json = response.json()
response_json

['Type',
 'Air temperature [K]',
 'Process temperature [K]',
 'Rotational speed [rpm]',
 'Torque [Nm]',
 'Tool wear [min]',
 'Difference temperature [K]',
 'Power']